# Demonstrations for structured data

In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv
import json

load_dotenv()

# Get the root directory of the project
root_dir = Path.cwd().parent.parent

## Open JSON files and create Pandas dataframe (WIP)

In [ ]:
import pandas as pd
import numpy as np

with open(root_dir / 'data' / 'ground-truth' / 'json' / 'gt_kbaa-p121.json', 'r') as file:
    file_json = json.loads(file.read())
    entries = file_json['entries']
    df = pd.DataFrame(entries)

df.head()

## Search Wikipedia for authors (WIP)

I am following [this page](https://api.wikimedia.org/wiki/Searching_for_Wikipedia_articles_using_Python) to search on Wikipedia

In [ ]:
import requests

# wikimedia_api_key = os.getenv('WIKIMEDIA_API_KEY')
def search_wikipedia(search_query, number_of_results=1):
    language_code = 'en'
    headers = {
    #   'Authorization': f'Bearer {wikimedia_api_key}',
        'User-Agent': 'Tim-KBAA yuyu.tim@gmail.com'
    }

    base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
    endpoint = '/search/page'
    url = base_url + language_code + endpoint
    parameters = {'q': search_query, 'limit': number_of_results}
    response = requests.get(url, headers=headers, params=parameters)

    return response

def formatted_names(lastname, firstname):
    """
    Create an array of potential names a person might be listed as. Assume last name is present.
    """

    # Replace empty names with None
    cleaned_lastname = lastname if lastname and not np.isnan(lastname) else None
    cleaned_firstname = firstname if firstname and not np.isnan(firstname) else None

    # WIP: handle suffixes? Only one ground truth JSON file has it.

    # Split given names (return empty list if no first name)
    split_names = str.split(cleaned_firstname, ' ') if cleaned_firstname else []

    # Create a list of a person's names in order (first name, middle name(s), last name)
    ordered_names = split_names.append(cleaned_lastname)
    initials = list(map(lambda name: name[0].upper(), ordered_names))
    initials_with_periods = list(map(lambda initial: initial + '.', initials))

    # Provide possible combinations of names (for searching purposes) and return
    if len(ordered_names) == 1:
        return [ordered_names[0]]
    if len(ordered_names) == 2:
        return [f'{ordered_names[0]} {ordered_names[1]}']
    else:
        name_combinations = [
            f'{ordered_names[0]} {ordered_names[-1]}',
            ' '.join(ordered_names),
            f'{ordered_names[0]} {' '.join(initials_with_periods[1:-1])} {ordered_names[-1]}',
            f'{' '.join(initials_with_periods[:-1])} {ordered_names[-1]}'
        ]
        return name_combinations

In [ ]:
for row in df.itertuples():
    formatted_name = f'{row.firstname} {row.lastname}'
    print(f'Searching Wikipedia for author {formatted_name}...')
    response = search_wikipedia(formatted_name)
    print(response.json())